# Trabajando con SQLite en Julia
Este notebook utiliza Julia para interactuar con SQLite y realizar operaciones en la base de datos `circuits.db`.

In [1]:
# Instalar el paquete SQLite.jl si no está instalado
using Pkg
Pkg.add("SQLite")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
Precompiling project...
           ✗ Adapt → AdaptStaticArraysExt
  0 dependencies successfully precompiled in 2 seconds. 446 already precompiled.


## Configurar SQLite en Julia
Conectaremos a la base de datos `circuits.db` y crearemos una tabla para almacenar los datos del archivo CSV.

In [2]:
using SQLite

# Conectar a la base de datos SQLite
db = SQLite.DB("circuits.db")

# Crear la tabla "circuits"
SQLite.execute(db, """
CREATE TABLE IF NOT EXISTS circuits (
    circuitId INTEGER,
    circuitRef TEXT,
    name TEXT,
    location TEXT,
    country TEXT,
    lat REAL,
    lng REAL,
    alt REAL,
    url TEXT
);
""")

println("Tabla circuits creada con éxito.")

Tabla circuits creada con éxito.


## Importar datos desde el archivo CSV
Leemos los datos de `circuits.csv` y los insertamos en la tabla SQLite.

In [3]:
using CSV
using DataFrames
using SQLite

# Leer el archivo CSV
data = CSV.read("circuits.csv", DataFrame)

# Insertar datos en la tabla circuits
for row in eachrow(data)
    try
        SQLite.execute(db, "INSERT INTO circuits VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
                       Tuple(row))
    catch err
        println("Error al insertar la fila: $row. Detalles: $err")
    end
end

println("Datos importados correctamente a la tabla circuits.")


Datos importados correctamente a la tabla circuits.


## Realizar consultas SQL
Ejecutaremos consultas en la tabla para verificar los datos.

In [4]:
using SQLite

# Conectar a la base de datos
db = SQLite.DB("circuits.db")

# Realizar la consulta SELECT * FROM circuits
query = DBInterface.execute(db, "SELECT * FROM circuits")
for row in query
    println(row)  # Esto imprime cada fila de la tabla circuits
end

# Cerrar la conexión
SQLite.close(db)






SQLite.Row{false}:
 :circuitId     1
 :circuitRef     "albert_park"
 :name           "Albert Park Grand Prix Circuit"
 :location       "Melbourne"
 :country        "Australia"
 :lat         -37.8497
 :lng         144.968
 :alt          10.0
 :url            "http://en.wikipedia.org/wiki/Melbourne_Grand_Prix_Circuit"
SQLite.Row{false}:
 :circuitId     2
 :circuitRef     "sepang"
 :name           "Sepang International Circuit"
 :location       "Kuala Lumpur"
 :country        "Malaysia"
 :lat           2.76083
 :lng         101.738
 :alt            missing
 :url            "http://en.wikipedia.org/wiki/Sepang_International_Circuit"
SQLite.Row{false}:
 :circuitId    3
 :circuitRef    "bahrain"
 :name          "Bahrain International Circuit"
 :location      "Sakhir"
 :country       "Bahrain"
 :lat         26.0325
 :lng         50.5106
 :alt           missing
 :url           "http://en.wikipedia.org/wiki/Bahrain_International_Circuit"
SQLite.Row{false}:
 :circuitId    4
 :circuitRef    "cata

#### Usando PretyTables
import Pkg; Pkg.add("PrettyTables")

In [5]:
using SQLite, PrettyTables

# Conectar a la base de datos
db = SQLite.DB("circuits.db")

# Ejecutar la consulta
query = DBInterface.execute(db, "SELECT * FROM circuits")

# Convertir los resultados a un DataFrame para manipulación más sencilla
using DataFrames
data = DataFrame(query)

# Mostrar la tabla ordenada
pretty_table(data)

# Cerrar la conexión
SQLite.close(db)


┌───────────┬────────────────┬───────────────────────────────────────────┬───────────────────────┬──────────────┬──────────┬───────────┬──────────┬─────────────────────────────────────────────────────────────────────────┐
│ circuitId │     circuitRef │                                      name │              location │      country │      lat │       lng │      alt │                                                                     url │
│     Int64 │         String │                                    String │                String │       String │  Float64 │   Float64 │ Float64? │                                                                  String │
├───────────┼────────────────┼───────────────────────────────────────────┼───────────────────────┼──────────────┼──────────┼───────────┼──────────┼─────────────────────────────────────────────────────────────────────────┤
│         1 │    albert_park │            Albert Park Grand Prix Circuit │             Melbourne │    Australia 

### Contar circuitos por país
Consulta para agrupar y contar el número de circuitos en cada país.

In [6]:
using SQLite, DataFrames, PrettyTables

# Conectar a la base de datos
db = SQLite.DB("circuits.db")

# Ejecutar la consulta para agrupar y contar circuitos por país
query = DBInterface.execute(db, """
SELECT country, COUNT(*) AS total_circuits
FROM circuits
GROUP BY country
ORDER BY total_circuits DESC;
""")

# Convertir los resultados a un DataFrame
data = DataFrame(query)

# Mostrar los datos en forma de tabla
pretty_table(data)

# Cerrar la conexión
SQLite.close(db)


┌──────────────┬────────────────┐
│      country │ total_circuits │
│       String │          Int64 │
├──────────────┼────────────────┤
│          USA │             77 │
│       France │             49 │
│        Spain │             42 │
│           UK │             28 │
│     Portugal │             21 │
│        Japan │             21 │
│        Italy │             21 │
│      Germany │             21 │
│       Canada │             21 │
│      Belgium │             21 │
│      Austria │             21 │
│ South Africa │             14 │
│       Brazil │             14 │
│    Australia │             14 │
│          UAE │              7 │
│       Turkey │              7 │
│  Switzerland │              7 │
│       Sweden │              7 │
│    Singapore │              7 │
│       Russia │              7 │
│  Netherlands │              7 │
│      Morocco │              7 │
│       Monaco │              7 │
│       Mexico │              7 │
│     Malaysia │              7 │
│        Korea

### Los circuitos con mayor altitud
Consulta para listar los cinco circuitos con mayor altitud.

In [7]:
using SQLite, DataFrames, PrettyTables

# Conectar a la base de datos
db = SQLite.DB("circuits.db")

# Ejecutar la consulta para obtener los cinco circuitos con mayor altitud
query = DBInterface.execute(db, """
SELECT name, country, alt
FROM circuits
ORDER BY alt DESC
LIMIT 5;
""")

# Convertir los resultados a un DataFrame
data = DataFrame(query)

# Mostrar los datos en forma de tabla
pretty_table(data)

# Cerrar la conexión
SQLite.close(db)


┌────────────────────────────────┬───────────┬─────────┐
│                           name │   country │     alt │
│                         String │    String │ Float64 │
├────────────────────────────────┼───────────┼─────────┤
│ Albert Park Grand Prix Circuit │ Australia │    10.0 │
│ Albert Park Grand Prix Circuit │ Australia │    10.0 │
│ Albert Park Grand Prix Circuit │ Australia │    10.0 │
│ Albert Park Grand Prix Circuit │ Australia │    10.0 │
│ Albert Park Grand Prix Circuit │ Australia │    10.0 │
└────────────────────────────────┴───────────┴─────────┘


### subqueries

El comando SELECT proporciona una gran flexibilidad, pero hay ocasiones en las que un único comando SELECT no puede expresar completamente una consulta. Para ayudar en estas situaciones, SQL admite subconsultas. Una subconsulta no es más que una declaración SELECT que está incrustada dentro de otra declaración SELECT. Las subconsultas también se conocen como sub-selects.

Las subconsultas se encuentran más comúnmente en la cláusula FROM, donde actúan como una tabla fuente calculada. Este tipo de subconsulta puede devolver cualquier número de filas o columnas, y es similar a crear una vista o ejecutar la consulta, registrar los resultados en una tabla temporal y luego hacer referencia a esa tabla en la consulta principal. La principal ventaja de usar una subconsulta en línea es que el optimizador de consultas puede fusionar la subconsulta en la declaración SELECT principal y analizar el problema completo, lo que a menudo conduce a un plan de consulta más eficiente.

Para usar una subconsulta en la cláusula FROM, simplemente enciérrala entre paréntesis. Las siguientes dos declaraciones producirán el mismo resultado:

In [8]:
SELECT * FROM TblA AS a JOIN TblB AS b;
SELECT * FROM TblA AS a JOIN (SELECT * FROM TblB) AS b;


UndefVarError: UndefVarError: `SELECT` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Las subconsultas también pueden aparecer en otros lugares, incluidas expresiones generales utilizadas en cualquier comando SQL. Los operadores EXISTS e IN utilizan subconsultas. De hecho, puedes usar una subconsulta en cualquier lugar donde una expresión espere una lista de valores literales (sin embargo, una subconsulta no puede generar una lista de identificadores).

In [9]:
using SQLite, DataFrames, PrettyTables

# Conectar a la base de datos
db = SQLite.DB("circuits.db")

# Ejecutar la consulta usando una subquery con WHERE
query = DBInterface.execute(db, """
SELECT name, country, alt
FROM circuits
WHERE country IN (
    SELECT country
    FROM circuits
    GROUP BY country
    HAVING AVG(alt) > 5.0
)
ORDER BY country, alt DESC;
""")

# Convertir el resultado de la consulta en un DataFrame
data = DataFrame(query)

# Mostrar los resultados en forma de tabla ordenada
pretty_table(data)

# Cerrar la conexión
SQLite.close(db)



┌────────────────────────────────┬───────────┬──────────┐
│                           name │   country │      alt │
│                         String │    String │ Float64? │
├────────────────────────────────┼───────────┼──────────┤
│ Albert Park Grand Prix Circuit │ Australia │     10.0 │
│ Albert Park Grand Prix Circuit │ Australia │     10.0 │
│ Albert Park Grand Prix Circuit │ Australia │     10.0 │
│ Albert Park Grand Prix Circuit │ Australia │     10.0 │
│ Albert Park Grand Prix Circuit │ Australia │     10.0 │
│ Albert Park Grand Prix Circuit │ Australia │     10.0 │
│ Albert Park Grand Prix Circuit │ Australia │     10.0 │
│        Adelaide Street Circuit │ Australia │  missing │
│        Adelaide Street Circuit │ Australia │  missing │
│        Adelaide Street Circuit │ Australia │  missing │
│        Adelaide Street Circuit │ Australia │  missing │
│        Adelaide Street Circuit │ Australia │  missing │
│        Adelaide Street Circuit │ Australia │  missing │
│        Adela

#### Uso de where

- Necesita instalar import Pkg; Pkg.add("Tables")

In [10]:
using SQLite, DataFrames, PrettyTables

# Conectar a la base de datos
db = SQLite.DB("circuits.db")

# Ejecutar la consulta SQL con la cláusula WHERE
query = DBInterface.execute(db, """
SELECT name, country, alt
FROM circuits
WHERE country = 'USA'
ORDER BY alt DESC;
""")

# Convertir el resultado de la consulta en un DataFrame
data = DataFrame(query)

# Mostrar los resultados en forma de tabla ordenada
pretty_table(data)

# Cerrar la conexión
SQLite.close(db)



┌─────────────────────────────────┬─────────┬─────────┐
│                            name │ country │     alt │
│                          String │  String │ Missing │
├─────────────────────────────────┼─────────┼─────────┤
│     Indianapolis Motor Speedway │     USA │ missing │
│          Phoenix street circuit │     USA │ missing │
│          Detroit Street Circuit │     USA │ missing │
│                       Fair Park │     USA │ missing │
│                      Long Beach │     USA │ missing │
│        Las Vegas Street Circuit │     USA │ missing │
│                    Watkins Glen │     USA │ missing │
│ Riverside International Raceway │     USA │ missing │
│   Sebring International Raceway │     USA │ missing │
│         Circuit of the Americas │     USA │ missing │
│    Port Imperial Street Circuit │     USA │ missing │
│     Indianapolis Motor Speedway │     USA │ missing │
│          Phoenix street circuit │     USA │ missing │
│          Detroit Street Circuit │     USA │ mi

In [11]:
using SQLite, DataFrames, PrettyTables

# Conectar a la base de datos
db = SQLite.DB("circuits.db")

# Ejecutar la consulta SQL con COUNT y SUM
query = DBInterface.execute(db, """
SELECT country, COUNT(*) AS total_circuits, SUM(alt) AS total_altitude
FROM circuits
GROUP BY country
ORDER BY total_circuits DESC;
""")

# Convertir el resultado de la consulta en un DataFrame
data = DataFrame(query)

# Mostrar los resultados en forma de tabla ordenada
pretty_table(data)

# Cerrar la conexión
SQLite.close(db)


┌──────────────┬────────────────┬────────────────┐
│      country │ total_circuits │ total_altitude │
│       String │          Int64 │       Float64? │
├──────────────┼────────────────┼────────────────┤
│          USA │             77 │        missing │
│       France │             49 │        missing │
│        Spain │             42 │        missing │
│           UK │             28 │        missing │
│     Portugal │             21 │        missing │
│        Japan │             21 │        missing │
│        Italy │             21 │        missing │
│      Germany │             21 │        missing │
│       Canada │             21 │        missing │
│      Belgium │             21 │        missing │
│      Austria │             21 │        missing │
│ South Africa │             14 │        missing │
│       Brazil │             14 │        missing │
│    Australia │             14 │           70.0 │
│          UAE │              7 │        missing │
│       Turkey │              7

## Cerrar la conexión
Es importante cerrar la conexión con la base de datos al terminar.

In [12]:
# Cerrar la conexión con la base de datos
SQLite.close(db)
println("Conexión cerrada.")

Conexión cerrada.
